In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F


StatementMeta(, 21a26a24-1045-4e61-8b22-f37217f98658, 3, Finished, Available)

In [2]:
%%sql

create table if not exists project_lakehouse.gold_product
(
    Product_ID long,
    Product_Category string,
    Product string,
    Create_TS timestamp,
    Modified_TS timestamp
)
using delta


StatementMeta(, 21a26a24-1045-4e61-8b22-f37217f98658, 4, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
Max_Date=spark.sql("""select coalesce(max(Modified_TS),"1900-01-01") from project_lakehouse.gold_product""").first()[0]
Max_Date

StatementMeta(, 21a26a24-1045-4e61-8b22-f37217f98658, 5, Finished, Available)

'1900-01-01'

In [4]:
df_bronze = spark.sql("""select distinct 
                        Product_category , 
                        Product 
                        from project_lakehouse.bronze_sales where Modified_TS>'{}' """.format(Max_Date))

StatementMeta(, 21a26a24-1045-4e61-8b22-f37217f98658, 6, Finished, Available)

In [6]:
Max_ID = spark.sql(""" select coalesce (max(Product_ID),0) from project_lakehouse.gold_product """).first()[0]
Max_ID

StatementMeta(, 21a26a24-1045-4e61-8b22-f37217f98658, 8, Finished, Available)

0

In [7]:
#Creating incremental ID for Product_ID in Product Table
df_final =  df_bronze.withColumn("Product_ID", monotonically_increasing_id()+Max_ID+1)

StatementMeta(, 21a26a24-1045-4e61-8b22-f37217f98658, 9, Finished, Available)

In [8]:
df_final.createOrReplaceTempView("ViewProduct")

StatementMeta(, 21a26a24-1045-4e61-8b22-f37217f98658, 10, Finished, Available)

In [9]:
%%sql
select * from ViewProduct limit 10

StatementMeta(, 21a26a24-1045-4e61-8b22-f37217f98658, 11, Finished, Available)

<Spark SQL result set with 10 rows and 3 fields>

In [11]:
%%sql
merge into project_lakehouse.gold_product as gp
using ViewProduct as vp
    on gp.Product=vp.Product and gp.Product_Category=vp.Product_Category
when matched then
update SET
    gp.Modified_TS = current_timestamp()
--- Not MATCHED    
when not matched then
insert
(
    gp.Product_ID,
    gp.Product_Category,
    gp.Product,
    gp.Create_TS,
    gp.Modified_TS
)
VALUES
(
    vp.Product_ID,
    vp.Product_Category,
    vp.Product,
    current_timestamp(),
    current_timestamp()
)

StatementMeta(, 21a26a24-1045-4e61-8b22-f37217f98658, 13, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>

In [12]:
%%sql 
select * from project_lakehouse.gold_product limit 10

StatementMeta(, 21a26a24-1045-4e61-8b22-f37217f98658, 14, Finished, Available)

<Spark SQL result set with 10 rows and 5 fields>